<a href="https://colab.research.google.com/github/eduardodut/Analise-tweets-ICD-Unifor/blob/master/AnaliseTweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Leitura dos dados

In [5]:
def downloadJsonFiles():
  # Fetch a single <1MB file using the raw GitHub URL.
  !curl --remote-name \
      -H 'Accept: application/vnd.github.v3.raw' \
      --location https://raw.githubusercontent.com/eduardodut/Analise-tweets-ICD-Unifor/master/Dados/LulaOficial.json


  !curl --remote-name \
      -H 'Accept: application/vnd.github.v3.raw' \
      --location https://raw.githubusercontent.com/eduardodut/Analise-tweets-ICD-Unifor/master/Dados/jairbolsonaro.json     

downloadJsonFiles()

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 47.6M  100 47.6M    0     0  54.9M      0 --:--:-- --:--:-- --:--:-- 54.8M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 25.9M  100 25.9M    0     0  52.1M      0 --:--:-- --:--:-- --:--:-- 52.1M


In [0]:
import pandas as pd

def carregarDados():
  import json
  import pandas as pd
  dataset_lula = pd.DataFrame()
  #tweetsLula = pd.read_json('/content/LulaOficial.json', lines = True)
  with open('/content/LulaOficial.json') as lula_file:
    dados = json.load(lula_file)
    dataset_lula = pd.DataFrame(dados)


  dataset_bolso = pd.DataFrame()
  #tweetsLula = pd.read_json('/content/LulaOficial.json', lines = True)
  with open('/content/jairbolsonaro.json') as bolso_file:
    dados = json.load(bolso_file)
    dataset_bolso = pd.DataFrame(dados)

  return dataset_bolso, dataset_lula

dataset_bolso, dataset_lula = carregarDados()


In [41]:
dataset_bolso.head(1)

,created_at,id,id_str,full_text,truncated,display_text_range,entities,source,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,in_reply_to_screen_name,user,geo,coordinates,place,contributors,is_quote_status,retweet_count,favorite_count,favorited,retweeted,lang,extended_entities,possibly_sensitive,quoted_status_id,quoted_status_id_str,quoted_status_permalink,quoted_status
0,2010-04-01 02:59:50,11402700435,11402700435,"EM ""DITADURA"" SEM PAREDÃO, ATÉ CHICO ALENCAR É...",False,"[0, 137]","{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""https://about.twitter.com/products/tw...",NaN,None,NaN,None,None,"{'id': 128372940, 'id_str': '128372940', 'name...",None,None,None,None,False,30,15,False,False,pt,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
import pickle
def cleaning_data(dataset):

  #realiza cópia do dataset
  dataset_temp = dataset.copy()
  
  #conversão da coluna 'created_at' para formato de data
  dataset_temp['created_at'] = dataset_temp['created_at'].astype('datetime64[ns]')

  #criação da coluna 'caracter_count' a partir da coluna 'display_text_range', que indica o tamanho do tweet
  dataset_temp['character_count'] = [(dataset_temp['display_text_range'][i][1]-dataset_temp['display_text_range'][i][0] + 1)for i in range(0,dataset_temp.shape[0])]

  #função que extrai dados do dicionário da coluna 'user' e transforma em uma coluna do dataset
  def extracao_dados_user(coluna):
    if coluna not in dataset_temp.columns:
      dataset_temp[coluna] = [pd.DataFrame(dataset_temp['user']).loc[i][0].get(coluna) for i in range(0,dataset_temp.shape[0])]

  #Extração de dados de seguidores a cada tweet emitido
  extracao_dados_user('followers_count')

  #Número de tweets favoritados pelo usuário no momento da emissão do tweet
  extracao_dados_user('favourites_count')

  #Número de usuários que são seguidos pelo usuário no momento da emissão do tweet
  extracao_dados_user('friends_count')

  def extracao_entities():
         
    coluna_hashtags = []
    coluna_user_mentions = []

    for i in range(0,dataset_temp.shape[0]):
      list_hashtags = []
      list_user_mention = []
      if dataset_temp.loc[i]['entities'].get("hashtags") != None:
        for j in range(0,len(dataset_temp.loc[i]['entities'].get("hashtags"))):
          list_hashtags.append(dataset_temp.loc[i]['entities'].get("hashtags")[j].get('text'))
      coluna_hashtags.append(list_hashtags)

      if dataset_temp.loc[i]['entities'].get("use_mentions") != None:
        for j in range(0,len(dataset_temp.loc[i]['entities'].get("user_mentions"))):
          list_user_mention.append(dataset_temp.loc[i]['entities'].get("user_mentions")[j].get('name'))
      coluna_user_mentions.append(list_user_mention)

    dataset_temp['hashtags'] = coluna_hashtags
    dataset_temp['user_mention'] = coluna_user_mentions

  extracao_entities()
  colunas_para_remocao = ['id_str', 'truncated', 'display_text_range', 'entities', 'lang',
                                   'source', 'in_reply_to_status_id','in_reply_to_status_id_str',
                                   'in_reply_to_user_id','in_reply_to_user_id_str', 'in_reply_to_screen_name',
                                   'user','geo','coordinates', 'place', 'contributors', 'is_quote_status',
                                   'possibly_sensitive', 'quoted_status_id','quoted_status_id_str',
                                  'quoted_status_permalink', 'extended_entities', 'quoted_status']
  dataset_temp = dataset_temp.drop(colunas_para_remocao, axis = 1)
  
  
  return dataset_temp

 


In [0]:
import pickle

lula_data = cleaning_data(dataset_lula)

bolso_data = cleaning_data(dataset_bolso)


with open("dados_lula.pickle", "wb") as file:
   pickle.dump(lula_data, file)

with open("dados_bolso.pickle", "wb") as file:
   pickle.dump(bolso_data, file)

In [198]:
print(dataset_bolso.columns)
print(bolso_data.columns)

Index(['created_at', 'id', 'id_str', 'full_text', 'truncated',
       'display_text_range', 'entities', 'source', 'in_reply_to_status_id',
       'in_reply_to_status_id_str', 'in_reply_to_user_id',
       'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'user', 'geo',
       'coordinates', 'place', 'contributors', 'is_quote_status',
       'retweet_count', 'favorite_count', 'favorited', 'retweeted', 'lang',
       'extended_entities', 'possibly_sensitive', 'quoted_status_id',
       'quoted_status_id_str', 'quoted_status_permalink', 'quoted_status'],
      dtype='object')
Index(['created_at', 'id', 'full_text', 'retweet_count', 'favorite_count',
       'favorited', 'retweeted', 'lang', 'extended_entities',
       'character_count', 'followers_count', 'favourites_count',
       'friends_count', 'hashtags', 'user_mention'],
      dtype='object')


0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
       ... 
6789    NaN
6790    NaN
6791    NaN
6792    NaN
6793    NaN
Name: quoted_status, Length: 6794, dtype: object

In [192]:
lula_data['full_text'].loc[lula_data['possibly_sensitive'] == True]

5183    Defesa identifica táticas de lawfare em denúnc...
6602    Lava Jato usa o sistema judiciário para fazer ...
Name: full_text, dtype: object

In [83]:
#dataset_bolso['user'] = pd.DataFrame(dataset_bolso['user'])
dataset_bolso['user'].loc[0]
# 'favourites_count'
# 'followers_count'
# 'friends_count'

{'can_media_tag': True,
 'contributors_enabled': False,
 'created_at': 'Wed Mar 31 23:13:44 +0000 2010',
 'default_profile': False,
 'default_profile_image': False,
 'description': 'Capitão do Exército Brasileiro, eleito 38° Presidente da República Federativa do Brasil. 🇧🇷',
 'entities': {'description': {'urls': []},
  'url': {'urls': [{'display_url': 'bolsonaro.com.br',
     'expanded_url': 'http://www.bolsonaro.com.br',
     'indices': [0, 23],
     'url': 'https://t.co/bqpNS6l0al'}]}},
 'favourites_count': 2393,
 'follow_request_sent': False,
 'followed_by': False,
 'followers_count': 5759089,
 'following': False,
 'friends_count': 394,
 'geo_enabled': False,
 'has_extended_profile': True,
 'id': 128372940,
 'id_str': '128372940',
 'is_translation_enabled': False,
 'is_translator': False,
 'lang': None,
 'listed_count': 5504,
 'location': 'Brasília, Brasil',
 'name': 'Jair M. Bolsonaro',
 'notifications': False,
 'profile_background_color': '175C17',
 'profile_background_image_url':

In [17]:
print(.isnull().sum())

0


In [0]:
list_keep = ['created_at', 'full_text', 'truncated',
       'display_text_range', 'entities', 'source', 'in_reply_to_status_id',
       'in_reply_to_status_id_str', 'in_reply_to_user_id',
       'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'user', 'geo',
       'coordinates', 'place', 'contributors', 'is_quote_status',
       'retweet_count', 'favorite_count', 'favorited', 'retweeted', 'lang',
       'extended_entities', 'possibly_sensitive', 'quoted_status_id',
       'quoted_status_id_str', 'quoted_status_permalink', 'quoted_status']

In [0]:
print(dados.head().loc[0,1].keys())
print(type(dados.transpose()))
print(dados.transpose()[0][0].keys())

dict_keys(['created_at', 'id', 'id_str', 'full_text', 'truncated', 'display_text_range', 'entities', 'source', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place', 'contributors', 'is_quote_status', 'retweet_count', 'favorite_count', 'favorited', 'retweeted', 'lang'])
<class 'pandas.core.frame.DataFrame'>
dict_keys(['created_at', 'id', 'id_str', 'full_text', 'truncated', 'display_text_range', 'entities', 'source', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place', 'contributors', 'is_quote_status', 'retweet_count', 'favorite_count', 'favorited', 'retweeted', 'lang'])


In [0]:
 print(dados.transpose()[0][0])

{'created_at': 'Fri Aug 17 16:23:04 +0000 2018', 'id': 1030490201460801541, 'id_str': '1030490201460801541', 'full_text': '"O presidente Lula vai dar um passeio nessa eleição. Ganha no primeiro turno" - @Haddad_Fernando, vice na chapa, em coletiva de imprensa sobre a decisão da @ONUBrasil que assegura o direito de Lula ser candidato.', 'truncated': False, 'display_text_range': [0, 212], 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [{'screen_name': 'Haddad_Fernando', 'name': 'Fernando Haddad', 'id': 354095556, 'id_str': '354095556', 'indices': [80, 96]}, {'screen_name': 'ONUBrasil', 'name': 'ONU Brasil', 'id': 57763444, 'id_str': '57763444', 'indices': [156, 166]}], 'urls': []}, 'source': '<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 2670726740, 'id_str': '2670726740', 'name

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
downloadJsonFiles()

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    14  100    14    0     0     36      0 --:--:-- --:--:-- --:--:--    35
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    14  100    14    0     0     43      0 --:--:-- --:--:-- --:--:--    43


In [0]:
# Import the necessary modules
from nltk.tokenize import regexp_tokenize
from nltk.tokenize import TweetTokenizer
# Use the TweetTokenizer to tokenize all tweets into one list
tknzr = TweetTokenizer()
all_tokens = [tknzr.tokenize(t) for t in tweets]
print(all_tokens)